In [13]:
import os
import sys

import constants
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
import pickle

In [14]:
repo_path = "../ProjectAeon/aeon_mecha"

loader = GenericLoader.from_filesystem(
    repo_path+"/aeon",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()

python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)

vectorstore = FAISS.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

with open("vectorstore.pkl", "wb") as f:
	pickle.dump(vectorstore, f)

In [ ]:
# from langchain.document_loaders import GitLoader

# loader = GitLoader(repo_path="../ProjectAeon/aeon_mecha") # You can specify a branch if you want
# data = loader.load()